In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [ ]:
load_dotenv()  # Load environment variables from .env file
model = ChatOpenAI()

In [ ]:
class BlogState(TypedDict):
    topic: str
    outline: str
    draft: str

In [ ]:
def generate_outline(state: BlogState) -> BlogState:
    
    # fetch the topic from the state
    topic = state['topic']

    # create the prompt 
    prompt = f"Create a detailed outline for a blog post about: {topic}"

    # get the outline from the LLM
    outline = model.invoke(prompt).content

    # update the state with the outline
    state['outline'] = outline
    return state

In [ ]:
def write_draft(state: BlogState) -> BlogState:
    
    # fetch the outline from the state
    topic = state['topic']
    outline = state['outline']

    # create the prompt 
    prompt = f"Write a detailed blog post based on the title - {topic} following outline:\n{outline}"

    # get the draft from the LLM
    draft = model.invoke(prompt).content

    # update the state with the draft
    state['draft'] = draft
    return state

In [ ]:
graph = StateGraph(BlogState)

# define nodes
graph.add_node("generate_outline", generate_outline)
graph.add_node("write_draft", write_draft)

# define edges
graph.add_edge(START, "generate_outline")   
graph.add_edge("generate_outline", "write_draft")
graph.add_edge("write_draft", END)

# compile the graph
workflow = graph.compile()


In [ ]:
initial_state = {'topic': 'The Future of Artificial Intelligence'}
final_state = workflow.invoke(initial_state)
print(final_state['draft'])  # Output the generated blog draft
